In [ ]:
import numpy as np
import laserhockey.laser_hockey_env as lh
import gymnasium as gym
from importlib import reload

In [ ]:
np.set_printoptions(suppress=True)

# Normal Game Play

In [ ]:
reload(lh)

In [ ]:
env = lh.LaserHockeyEnv()

have a look at the initialization condition: alternating who starts and are random in puck position

In [ ]:
obs, info = env.reset()
obs_agent2 = env.obs_agent_two()
env.render()

one episode with random agents

In [ ]:
#obs = env.reset()
obs_agent2 = env.obs_agent_two()

for _ in range(600):
    env.render()
    a1 = np.random.uniform(-1,1,3)
    a2 = np.random.uniform(-1,1,3)    
    obs, r, d, _, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    if d: break

Without rendering, it runs much faster

In [ ]:
obs, info  = env.reset()
obs_agent2 = env.obs_agent_two()

for _ in range(600):    
    a1 = [1,-.5,0] # np.random.uniform(-1,1,3)
    a2 = [1,0.,0] # np.random.uniform(-1,1,3)*0    
    obs, r, d, _, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    if d: break

"info" dict contains useful proxy rewards and winning information

In [ ]:
info

Winner == 0: draw

Winner == 1: you (left player)

Winner == -1: opponent wins (right player)

# Train Shooting

In [ ]:
env = lh.LaserHockeyEnv(mode=lh.LaserHockeyEnv.TRAIN_SHOOTING)

In [ ]:
o, info = env.reset()
env.render()

In [ ]:
for _ in range(200):
    env.render()
    a1 = [1,0,0] # np.random.uniform(-1,1,3)
    a2 = [0,0.,0] 
    obs, r, d, _, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    if d: break

# Train DEFENDING

In [ ]:
reload(lh)

In [ ]:
env = lh.LaserHockeyEnv(mode=lh.LaserHockeyEnv.TRAIN_DEFENSE)

In [ ]:
o, info = env.reset()
env.render()

In [ ]:
for _ in range(60):
    env.render()
    a1 = [1,0,0] # np.random.uniform(-1,1,3)
    a2 = [0,0.,0] 
    obs, r, d, _, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    if d: break

# Using discrete actions

In [ ]:
env = lh.LaserHockeyEnv(mode=lh.LaserHockeyEnv.TRAIN_SHOOTING)

In [ ]:
import random

In [ ]:
for _ in range(200):
    env.render()
    a1_discrete = random.randint(0,7)
    a1 = env.discrete_to_continous_action(a1_discrete)
    a2 = [0,0.,0] 
    obs, r, d, _, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    if d: break

# Hand-crafted Opponent

In [ ]:
reload(lh)

In [ ]:
env = lh.LaserHockeyEnv()

In [ ]:
o, info = env.reset()
env.render()

In [ ]:
player1 = lh.BasicOpponent()
player2 = lh.BasicOpponent()

In [ ]:
obs_buffer = []

In [ ]:
obs, info = env.reset()
obs_agent2 = env.obs_agent_two()
for _ in range(100):
    env.render()
    a1 = player1.act(obs)
    a2 = player2.act(obs_agent2)
    obs, r, d, _, info = env.step(np.hstack([a1,a2]))    
    obs_buffer.append(obs)
    obs_agent2 = env.obs_agent_two()
    if d: break

In [ ]:
obs_buffer = np.asarray(obs_buffer)

In [ ]:
np.mean(obs_buffer,axis=0)

In [ ]:
np.std(obs_buffer,axis=0)

In [ ]:
scaling = [ 1.0,  1.0 , 3.14, 4.0, 4.0, 2.0,  
            1.0,  1.0,  3.14, 4.0, 4.0, 2.0,  
            2.0, 2.0, 10.0, 10.0]

In [ ]:
env.close()

# Human Opponent

In [ ]:
import time

In [ ]:
env = lh.LaserHockeyEnv()

In [ ]:
o, info = env.reset()
env.render()

In [ ]:
from laserhockey.hockey_env import HumanOpponent

In [ ]:
player1 = HumanOpponent(env=env, player=1)
player2 = lh.BasicOpponent()

In [ ]:
obs, info = env.reset()
time.sleep(1)
obs_agent2 = env.obs_agent_two()
for _ in range(1600):
    time.sleep(0.2)
    env.render()
    a1 = player1.act(obs)
    a2 = player2.act(obs_agent2)
    obs, r, d, _, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    if d: break